In [4]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

#r "nuget: Azure.AI.OpenAI"
#r "nuget: Azure.Identity"
#r "nuget: Azure"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Data.SqlClient"

Installed Packages Azure, 1.0.6 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.14.0 Microsoft.Data.SqlClient, 6.0.2 Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25177.1 Newtonsoft.Json, 13.0.3

Loading extension script from `C:\Users\hasan\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25177.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [7]:
#!connect mssql --kernel-name Stackoverflow --connection-string "Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;""

Kernel added: #!sql-Stackoverflow

In [ ]:
ALTER TABLE Posts ADD BodyVector VECTOR(1536)

In [12]:
SELECT TOP 1 PostId, Title, PostBody, ViewCount, PostType, OwnerUserId, Score
 FROM Posts WHERE PostType = 'Question'

(1 row affected)

PostId Title PostBody ViewCount PostType OwnerUserId Score 20047 Diagnosing Deadlocks in SQL Server 2005 <p>We're seeing some pernicious, but rare, deadlock conditions in the Stack Overflow SQL Server 2005 database.</p>

<p>I attached the profiler, set up a trace profile using <a href="http://www.simple-talk.com/sql/learn-sql-server/how-to-track-down-deadlocks-using-sql-server-2005-profiler/" rel="noreferrer">this excellent article on troubleshooting deadlocks</a>, and captured a bunch of examples. The weird thing is that <strong>the deadlocking write is <em>always</em> the same</strong>:</p>

<pre><code>UPDATE [dbo].[Posts]
SET [AnswerCount] = @p1, [LastActivityDate] = @p2, [LastActivityUserId] = @p3
WHERE [Id] = @p0
</code></pre>

<p>The other deadlocking statement varies, but it's usually some kind of trivial, simple <strong>read</strong> of the posts table. This one always gets killed in the deadlock. Here's an example</p>

<pre><code>SELECT
[t0].[Id], [t0].[PostTypeId], [t0].[Score], [t0].[Views], [t0].[AnswerCount], 
[t0].[AcceptedAnswerId], [t0].[IsLocked], [t0].[IsLockedEdit], [t0].[ParentId], 
[t0].[CurrentRevisionId], [t0].[FirstRevisionId], [t0].[LockedReason],
[t0].[LastActivityDate], [t0].[LastActivityUserId]
FROM [dbo].[Posts] AS [t0]
WHERE [t0].[ParentId] = @p0
</code></pre>

<p>To be perfectly clear, we are not seeing write / write deadlocks, but read / write.</p>

<p>We have a mixture of LINQ and parameterized SQL queries at the moment. We have added <code>with (nolock)</code> to all the SQL queries. This may have helped some. We also had a single (very) poorly-written badge query that I fixed yesterday, which was taking upwards of 20 seconds to run every time, and was running every minute on top of that. I was hoping this was the source of some of the locking problems!</p>

<p>Unfortunately, I got another deadlock error about 2 hours ago. Same exact symptoms, same exact culprit write.</p>

<p>The truly strange thing is that the locking write SQL statement you see above is part of a very specific code path. It's <em>only</em> executed when a new answer is added to a question -- it updates the parent question with the new answer count and last date/user. This is, obviously, not that common relative to the massive number of reads we are doing! As far as I can tell, we're not doing huge numbers of writes anywhere in the app.</p>

<p>I realize that NOLOCK is sort of a giant hammer, but most of the queries we run here don't need to be that accurate. Will you care if your user profile is a few seconds out of date?</p>

<p>Using NOLOCK with Linq is a bit more difficult as <a href="http://www.hanselman.com/blog/GettingLINQToSQLAndLINQToEntitiesToUseNOLOCK.aspx" rel="noreferrer">Scott Hanselman discusses here</a>.</p>

<p>We are flirting with the idea of using</p>

<pre><code>SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
</code></pre>

<p>on the base database context so that all our LINQ queries have this set. Without that, we'd have to wrap every LINQ call we make (well, the simple reading ones, which is the vast majority of them) in a 3-4 line transaction code block, which is ugly.</p>

<p>I guess I'm a little frustrated that trivial reads in SQL 2005 can deadlock on writes. I could see write/write deadlocks being a huge issue, but <em>reads?</em> We're not running a banking site here, we don't need perfect accuracy every time.</p>

<p>Ideas? Thoughts?</p>

<hr>

<blockquote>
 <p>Are you instantiating a new LINQ to SQL DataContext object for every operation or are you perhaps sharing the same static context for all your calls?</p>
</blockquote>

<p>Jeremy, we are sharing one static datacontext in the base Controller for the most part:</p>

<pre><code>private DBContext _db;
/// &lt;summary&gt;
/// Gets the DataContext to be used by a Request's controllers.
/// &lt;/summary&gt;
public DBContext DB
{
 get
 {
 if (_db == null)
 {
 _db = new DBContext() { SessionName = GetType().Name };
 //_db.ExecuteCommand("SET TRANS

In [14]:

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using OpenAI.Embeddings;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Net.Http;
using System.Collections.ObjectModel;
using Microsoft.Data.SqlClient;


In [15]:
var openAIClient = new AzureOpenAIClient(
    new Uri("https://savranweb.openai.azure.com/"),
    new AzureKeyCredential("3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ"));
var aiclient = openAIClient.GetEmbeddingClient("embedding");

In [16]:
ReadOnlyMemory<float> GenerateVector(string text)
{    
    OpenAIEmbedding newembedding = aiclient.GenerateEmbedding(text);
    return newembedding.ToFloats();
}

In [17]:
public class Post    
{    
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int OwnerUserId { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public float[] bodyvector {get;set;}
}

In [ ]:
var json = await new HttpClient().GetStringAsync("https://raw.githubusercontent.com/hsavran/Presentations/refs/heads/main/stackoverflow.json");
var postList = JsonConvert.DeserializeObject<List<Post>>(json);
postList.Count.Display();
var inserttable = new System.Data.DataTable("Posts");
inserttable.Columns.Add("PostId", typeof(int));
inserttable.Columns.Add("Title", typeof(string));
inserttable.Columns.Add("PostBody", typeof(string));
inserttable.Columns.Add("ViewCount", typeof(int));
inserttable.Columns.Add("PostType", typeof(string));
inserttable.Columns.Add("OwnerUserId", typeof(int));
inserttable.Columns.Add("Score", typeof(int));
inserttable.Columns.Add("bodyvector", typeof(string));

foreach (var post in postList.Where(p => p.PostType == "Question" && p.Title.Contains("SQL")).Take(1000))
{   
    var newRow = inserttable.NewRow();
    newRow["PostId"] = post.PostId;
    newRow["PostBody"] = post.PostBody;
    newRow["Title"] = post.Title;
    newRow["ViewCount"] = post.ViewCount;
    newRow["OwnerUserId"] = post.OwnerUserId;
    newRow["PostType"] = post.PostType;
    newRow["Score"] = post.Score;
    var temp = GenerateVector(post.PostBody).ToArray();
    var json = JsonConvert.SerializeObject(temp);
    newRow["bodyvector"] = json;
    //JsonConvert.SerializeObject(string.Join(",", temp));
    inserttable.Rows.Add(newRow);
}
//inserttable.Rows.Count.Display();
//inserttable.Rows[0].Display();
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var transaction = connection.BeginTransaction())
    {
        using (var bulkCopy = new Microsoft.Data.SqlClient.SqlBulkCopy(connection, Microsoft.Data.SqlClient.SqlBulkCopyOptions.Default, transaction))
        {
            bulkCopy.DestinationTableName = "Posts";
            bulkCopy.BatchSize = 1000;
            bulkCopy.WriteToServer(inserttable);
        }
        transaction.Commit();
    }
}

In [18]:
var vectortosearch = GenerateVector("What are the most common questions about relational databases").ToArray();
var temp = JsonConvert.SerializeObject(vectortosearch);
// run the query to get the top 10 most similar posts
var cmd = @"
DECLARE @v AS VECTOR(1536);
--DECLARE @temp AS NVARCHAR(MAX);
SET @v = (SELECT CAST(@temp AS VECTOR(1536)))
SELECT TOP(10)   
  title,
  VECTOR_DISTANCE('cosine', @v, bodyvector) AS distance
FROM 
  Posts
ORDER BY
  distance desc";
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var command = new Microsoft.Data.SqlClient.SqlCommand(cmd, connection))
    {
        command.Parameters.AddWithValue("@temp", temp);
        using (var reader = command.ExecuteReader())
        {
            while (reader.Read())
            {
                Console.WriteLine($"Title: {reader["title"]}, Distance: {reader["distance"].ToString()}");               
              
            }
        }
    }
}

Title: What is best practice for FTP from a SQL Server 2005 stored procedure?, Distance: 0.3161234022465138
Title: Decoding T-SQL CAST in C#/VB.NET, Distance: 0.31561580662175304
Title: Using Parameters in MS Reporting Services (SQL Server 2008) against an ODBC data source, Distance: 0.3117094200667455
Title: SQL 2005 Reporting Services custom report item (CRI) - what are the limits?, Distance: 0.3108764845457265
Title: What is the best method for checking if a file exists from a SQL Server 2005 stored procedure?, Distance: 0.30944626746173876
Title: Best method for varchar date validation in Sybase (T-SQL)?, Distance: 0.3079388520235
Title: Using cached credentials to connect to SQL 2005 across a domain boundary, Distance: 0.304684993515298
Title: Conditional Visibility and Page Breaks with SQL Server 2005 Reporting Services, Distance: 0.3034064229479754
Title: Oracle SQL Developer not responsive when trying to view tables (or suggest an Oracle Mac client), Distance: 0.303236342609665